require input "X_train_rotate.npy"
"X_test.npy"
"target_train.npy"
"test_id"

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import optimizers
from keras.optimizers import Adam, SGD
from keras.applications.vgg19 import VGG19

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
epochs = 30
batch_size = 64
split = 0.9

In [3]:
X_train_rotate=np.load('X_train_rotate.npy')
target_train=np.load('target_train.npy')
X_test=np.load('X_test.npy')

In [4]:
def get_callbacks(filepath, patience):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [5]:
adam=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

def train_top_model(input_shape):
    
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=adam,
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    #model.summary()
    return model
    
    

In [6]:
file_path = 'top_model_weights_path.h5'
#callbacks = get_callbacks(file_path,patience=5)

In [7]:
import os 
model = applications.VGG16(include_top=False, weights='imagenet')

#bottleneck_features_validation = model.predict(X_test)
#np.save('bottleneck_features_test.npy', bottleneck_features_validation)


X_train = model.predict(X_train_rotate)
input_shape=X_train.shape[1:]

#5fold cross validation
from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=8, shuffle=False, random_state=seed)

cvscores = []

model = train_top_model(input_shape)
for train, test in kfold.split(X_train, target_train):
    model = train_top_model(input_shape)
    model.fit(X_train[train], target_train[train],
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=0,
                    validation_data=(X_train[test], target_train[test]),
                    callbacks=get_callbacks(file_path, patience=5))
    model.load_weights(filepath=file_path)
    scores = model.evaluate(X_train[test], target_train[test], verbose=0)
    print(model.metrics_names[0], scores[0])
    cvscores.append(scores[0])
    

loss 0.342865028883
loss 0.309407920734
loss 0.320067692239
loss 0.348974699387
loss 0.313500941246
loss 0.377066934763
loss 0.302104912463
loss 0.362496999209


In [8]:
train.shape

(4211,)

In [9]:
print('Test loss average:',np.mean(cvscores), 'Test loss std:',np.std(cvscores))

Test loss average: 0.334560641116 Test loss std: 0.025503782136


In [10]:
'from matplotlib import pyplot as plt
%matplotlib inline
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

SyntaxError: EOL while scanning string literal (<ipython-input-10-21d7bbece636>, line 1)

In [ ]:
X_test = np.load('bottleneck_features_test.npy')
predicted_test=model.predict(X_test,verbose=1)

In [ ]:
import pandas as pd
submission = pd.DataFrame()
test_id=np.load('test_id.npy')
submission['id']=test_id
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)